In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *

### Example of posterior computation

In [2]:
compiledFile=compile2SOGA('../programs/SOGA/BernoulliTorch.soga')
cfg = produce_cfg(compiledFile)
output_dist = start_SOGA(cfg)

EntryNode<> Dist<['theta', 'y'],pi: [1.0]  mu: [tensor([0., 0.], requires_grad=True)] sigma: [tensor([[1.0000e-08, 0.0000e+00],
        [0.0000e+00, 1.0000e-08]])]>
True
StateNode<state0,None,theta=gm([0.5, 0.5],[0.25, 0.75],[0.1443, 0.1443])> Dist<['theta', 'y'],pi: [1.0]  mu: [tensor([0., 0.], requires_grad=True)] sigma: [tensor([[1.0000e-08, 0.0000e+00],
        [0.0000e+00, 1.0000e-08]])]>
True
TestNode<test0,gm([0.5, 0.5],[0.25, 0.75],[0.1443, 0.1443])-theta<0> Dist<['theta', 'y'],pi: [0.5, 0.5]  mu: [tensor([0.2500, 0.0000], grad_fn=<CopySlices>), tensor([0.7500, 0.0000], grad_fn=<CopySlices>)] sigma: [tensor([[2.0822e-02, 0.0000e+00],
        [0.0000e+00, 1.0000e-08]]), tensor([[2.0822e-02, 0.0000e+00],
        [0.0000e+00, 1.0000e-08]])]>
True
StateNode<state1,True,y=gm([1.],[0.],[0.01])> Dist<['theta', 'y'],pi: [0.5, 0.5]  mu: [tensor([0.2500, 0.0000], grad_fn=<CopySlices>), tensor([0.7500, 0.0000], grad_fn=<CopySlices>)] sigma: [tensor([[2.0822e-02, 0.0000e+00],
        [0.00

In [3]:
output_dist.gm.marg_pdf(torch.tensor(0.001), 1)

tensor([19.8476], grad_fn=<AddBackward0>)

### Example of gradient-based optimization

In [2]:
compiledFile=compile2SOGA('../programs/SOGA/Optim.soga')
cfg = produce_cfg(compiledFile)

In [3]:
def compute_posterior(init_mean):
    # initializes current_dist
    var_list = cfg.ID_list
    data = cfg.data
    gm = GaussianMix([torch.tensor(1.)], [init_mean], [torch.eye(len(var_list))])
    init_dist = Dist(var_list, gm)
    cfg.root.set_dist(init_dist)

    # initializes visit queue
    exec_queue = [cfg.root]

    # executes SOGA on nodes on exec_queue
    while(len(exec_queue)>0):
        SOGA(exec_queue.pop(0), data, False, exec_queue)

    return cfg.node_list['exit'].dist
    
    # returns output distribution
    #p, current_dist = merge(cfg.node_list['exit'].list_dist)
    #cfg.node_list['exit'].list_dist = []
    #return current_dist

def compute_posterior_dummy(init_mean):
    var_list = cfg.ID_list
    dist = Dist(var_list, GaussianMix([1.], [init_mean], [torch.eye(len(var_list))]))
    new_dist = update_rule(dist, 'm = gm([1.], [0.], [1.])', {})
    #new_dist = update_rule(dist, 'skip', {})
    return new_dist

In [4]:
init_mean = torch.tensor([0.0], requires_grad=True)  # Starting point for init_mean
init_mean = torch.nn.Parameter(init_mean)

entry_dist = compute_posterior(init_mean)
entry_dist

Dist<['m'],pi: [tensor(1.)]  mu: [tensor([0.], grad_fn=<CloneBackward0>)] sigma: [tensor([[2.]])]>

In [10]:
samples = distributions.Normal(torch.tensor(5.), torch.tensor(1.)).sample((100,1))

In [11]:
def neg_likelihood(dist, samples):
    likelihood = torch.tensor([0.])
    for sample in samples:
        likelihood += current_dist.gm.marg_pdf(sample,0)
    return -likelihood

In [14]:
import torch

# Ensure init_mean is a Parameter so it can be optimized
init_mean = torch.tensor([0.0], requires_grad=True)  # Starting point for init_mean
init_mean = torch.nn.Parameter(init_mean)

# Define the optimizer with init_mean as the parameter
optimizer = torch.optim.SGD([init_mean], lr=0.1)

for i in range(50):
    optimizer.zero_grad()  # Reset gradients
    
    # loss
    current_dist = compute_posterior(init_mean)
    loss = neg_likelihood(current_dist, samples)
    
    # Backpropagate
    loss.backward()
    
    # Update init_mean
    optimizer.step()
    
    # Print progress
    if i % 10 == 0:
        print(f"x: {init_mean.item()}, loss: {loss.item()}")

x: 0.036618512123823166, loss: -0.18703266978263855
x: 0.6156790256500244, loss: -0.49764034152030945
x: 3.7490010261535645, loss: -10.335077285766602
x: 5.0711774826049805, loss: -23.728288650512695
x: 5.0711774826049805, loss: -23.728290557861328
